## Question  3 - Importing Libraries and Data

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Importing Data
path = r'C:\Users\javir\Downloads\Javis Data Analytics stuff\Achievement 4\Instacart Basket Analysis'
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))
ords_prods_merg = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merg_flag.pkl'))

## Question 4 - Wrangle customer data

In [3]:
# Checking our columns and the values stored within those columns
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### Renaming columns

In [12]:
# Changing these columns to fit a uniform look for our columns
cust.rename(columns = {'First Name' : 'first_name'}, inplace = True)
cust.rename(columns = {'Surnam' : 'last_name'}, inplace = True)
cust.rename(columns = {'Gender' : 'gender'}, inplace = True)
cust.rename(columns = {'STATE' : 'state'}, inplace = True)
cust.rename(columns = {'Age' : 'age'}, inplace = True)
# Changing these columns for clearer representation of what they stand for
cust.rename(columns = {'n_dependants' : 'num_of_dependants'}, inplace = True)
cust.rename(columns = {'fam_status' : 'family_status'}, inplace = True)

#### We aren't going to be dropping any columns as we don't know what kind of questions we are looking to answer. Keeping the data ensures we will have all tools available to answer questions if they arise.

## Question 5 - Data quality check

#### Checklist:
#### - Missing Values
#### - Duplicate Values
#### - Mixed-type data column

In [13]:
# Checking row counts
cust.shape

(206209, 10)

In [14]:
# Checking statistical info about our data set
cust.describe()

,user_id,age,num_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [15]:
# Checking dtyoes for columns
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int64 
 1   first_name         194950 non-null  object
 2   last_name          206209 non-null  object
 3   gender             206209 non-null  object
 4   state              206209 non-null  object
 5   age                206209 non-null  int64 
 6   date_joined        206209 non-null  object
 7   num_of_dependants  206209 non-null  int64 
 8   family_status      206209 non-null  object
 9   income             206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


#### - Missing Values

In [16]:
# Checking null values
cust.isnull().sum()

user_id                  0
first_name           11259
last_name                0
gender                   0
state                    0
age                      0
date_joined              0
num_of_dependants        0
family_status            0
income                   0
dtype: int64

In [17]:
# Located 11,259 rows of null values in first_name, creating a table inlcuding ONLY null values
cust_nan = cust[cust['first_name'].isnull() == True]

In [18]:
# I wont be removing these 11,259 rows, we still hold all the information about their account exluding their first name, the information can still be valuable
cust_nan.head()

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_dependants,family_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819


#### - Duplicate Values

In [23]:
# Checkign for duplicate rows within our dataframe
cust[cust.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_dependants,family_status,income


#### - Mixed data-types

In [24]:
# Checking for mixed values
for col in cust.columns.tolist():
    weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cust[weird]) > 0:
        print(col)

first_name


#### first_name column has mixed data, however as we saw earlier in .info() first_name is an dtype(object), indicating it is already set in string format

In [33]:
cust['first_name'] = cust['first_name'].astype('str')

In [34]:
# Confirmation of first_name being an object data type, executing code above did not alter dtype
cust['first_name'].dtype

dtype('O')

## Question 6 - Combine Customer Data with ords_prods_merge

In [38]:
pd.set_option("display.max_columns", None)
# Data set ords_prods_merge has already been imported
# Checking both data frames columns for potential keys
cust

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799


In [39]:
ords_prods_merg

,order_id,user_id,evaluation_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_day,busiest_days,busiest_days_revised,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spender_flag,order_frequency,frequency_flag
0,2539329,1,prior,1,2,8,NaN,True,196.0,1.0,0.0,Soda,77.0,7.0,9.0,both,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer
1,2539329,1,prior,1,2,8,NaN,True,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer
2,2539329,1,prior,1,2,8,NaN,True,12427.0,3.0,0.0,Original Beef Jerky,23.0,19.0,4.4,both,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer
3,2539329,1,prior,1,2,8,NaN,True,26088.0,4.0,0.0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer
4,2539329,1,prior,1,2,8,NaN,True,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32641263,2977660,206209,prior,13,1,12,7.0,False,38730.0,6.0,0.0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,both,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer
32641264,2977660,206209,prior,13,1,12,7.0,False,31477.0,7.0,0.0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,both,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer
32641265,2977660,206209,prior,13,1,12,7.0,False,6567.0,8.0,0.0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,both,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer
32641266,2977660,206209,prior,13,1,12,7.0,False,22920.0,9.0,0.0,Roasted & Salted Shelled Pistachios,117.0,19.0,1.7,both,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer


In [40]:
# Dropping '_merge' column, so we can check if we got a full match, and we cannot .merge() with indicator = True if '_merge' already exists
merg_final = ords_prods_merg.drop(columns = '_merge')

In [42]:
# Both data frames sharea common column 'user_id' which will be our key for the merge
combined = merg_final.merge(cust, on = 'user_id', how = 'left', indicator = True )

In [44]:
combined

,order_id,user_id,evaluation_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,busiest_day,busiest_days,busiest_days_revised,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spender_flag,order_frequency,frequency_flag,first_name,last_name,gender,state,age,date_joined,num_of_dependants,family_status,income,_merge
0,2539329,1,prior,1,2,8,NaN,True,196.0,1.0,0.0,Soda,77.0,7.0,9.0,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,prior,1,2,8,NaN,True,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2539329,1,prior,1,2,8,NaN,True,12427.0,3.0,0.0,Original Beef Jerky,23.0,19.0,4.4,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2539329,1,prior,1,2,8,NaN,True,26088.0,4.0,0.0,Aged White Cheddar Popcorn,23.0,19.0,4.7,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,2539329,1,prior,1,2,8,NaN,True,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Regularly busy,Regularly busy,Regularly busy,Fewest Orders,11,Regular Customer,6.367797,Low spender,20.0,Frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32641263,2977660,206209,prior,13,1,12,7.0,False,38730.0,6.0,0.0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer,Diana,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
32641264,2977660,206209,prior,13,1,12,7.0,False,31477.0,7.0,0.0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer,Diana,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
32641265,2977660,206209,prior,13,1,12,7.0,False,6567.0,8.0,0.0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer,Diana,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
32641266,2977660,206209,prior,13,1,12,7.0,False,22920.0,9.0,0.0,Roasted & Salted Shelled Pistachios,117.0,19.0,1.7,Regularly busy,Regularly busy,Busiest days,Most Orders,14,Regular Customer,7.058915,Low spender,22.0,Frequent customer,Diana,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both


In [45]:
combined['_merge'].value_counts()

both          32641268
left_only            0
right_only           0
Name: _merge, dtype: int64

## Question 8 - Export

In [43]:
combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'cust_combined.pkl'))